# Intercycle Project: Artificial Intelligence

Carlos Cardoso A - Edith Guaraca C

In [1]:
import re
import itertools

class Utilities:
    
    def __init__(self, path = './corpus/digits-database.data'):
        self.path = path
        self.regex = re.compile('(0|1){2,}') # Busca patrones que coincidan con 2 o mas ceros o unos.
        self.regexno = re.compile('(\s)+[0-9]{1}') # Busca un unico numero el cual tenga un espacio o tabulacion antes del mismo.
        
    
    def generate_indices(self):
        _dict = []
        with open(self.path, 'r') as _f:
            pivote = 0
            flag = False
            lineno = 0
            for line in _f:
                if self.regex.match(line)!=None and not flag:
                    pivote = lineno
                    flag = True
                if self.regexno.match(line)!=None and flag:
                    _dict.append((int(line.replace(' ','')),pivote,lineno))
                    flag = False
                lineno += 1
            _f.close()
            
        return _dict

    def get_digit(self,_slice, _end):
        data = []
        with open(self.path, 'r') as _f:
            for line in itertools.islice(_f, _slice, _end):
                data.append([int(i) for i in line.lstrip().rstrip()])
            
            _f.close()
        return data

In [2]:
import numpy as np

class Converter:

	def __init__(self, lim = 946):
		self.utilities = Utilities()
		self.lim = lim
		self.indices = self.utilities.generate_indices()
		self.n = []
		self.inicioFin = []
		self.datos = []
		self.delta = []

	def formater(self):
		for j, k, l in self.indices:
			self.n.append(j)
			self.inicioFin.append((k,l))

		for i in range(0, self.lim):
			inicio, fin = self.inicioFin[i]
			fila = np.ravel(np.matrix(self.utilities.get_digit(inicio, fin)))
			self.datos.append(fila)
			self.delta.append(self.n[i])
		return (self.datos, self.delta)

Defining scikit learn, it is a free software machine learning library for the Python programming language. It features various classification, regression and clustering algorithms including support vector machines, random forests, gradient boosting, k-means and DBSCAN, and is designed to interoperate with the Python numerical and scientific libraries NumPy and SciPy.

Prerequisites required before using sklearn library

The library is built upon the SciPy (Scientific Python) that must be installed before you can use scikit-learn. This stack that includes:

· NumPy: Base n-dimensional array package

· SciPy: Fundamental library for scientific computing

· Matplotlib: Comprehensive 2D/3D plotting

· IPython: Enhanced interactive console

· Sympy: Symbolic mathematics

· Pandas: Data structures and analysis

In [3]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import numpy as np

class Red:

	def __init__(self, lim = 946, iteraciones = 100):
		self.converter = Converter(lim)
		self.data = self.converter.formater()
		self.label_encoder = LabelEncoder()
		self.datos, self.delta = self.data
		self.iteraciones = iteraciones
		self.codificar()
	
	def codificar(self):
		salida = self.label_encoder.fit_transform(self.delta)
		onehot_encoder = OneHotEncoder(sparse=False)
		salida = salida.reshape(len(salida), 1)
		self.onehot_encoded = onehot_encoder.fit_transform(salida)
	
	def entrenar(self):
		x_train, x_test, d_train, d_test = train_test_split(self.datos, self.onehot_encoded, test_size=0.80, random_state=0)
		self.mlp = MLPClassifier(solver = 'lbfgs', activation='logistic', verbose=True, alpha=1e-4, tol=1e-15, max_iter=self.iteraciones, \
		hidden_layer_sizes=(1024, 512, 256, 128, 10))
		self.mlp.fit(self.datos, self.onehot_encoded)
		
		self.predecir(self.datos[945])
		prediccion = (np.argmax(self.mlp.predict(x_test), axis = 1) + 1).reshape(-1, 1)
		print('Matriz de Confusion\n')
		matriz = confusion_matrix((np.argmax(d_test, axis = 1) + 1).reshape(-1, 1), prediccion)
		print(matriz)
		print('\n')
		print(classification_report((np.argmax(d_test, axis = 1) + 1).reshape(-1, 1), prediccion))

	def predecir(self, entrada):
		res = self.mlp.predict(entrada.reshape(1, -1))
		num = (np.argmax(res, axis=1)+1).reshape(-1, 1) # Decodifica el numero.
		aux = []
		matriz = []
		for i in range(32):
			for j in range(i * 32, (i + 1) * 32):
				aux.append(entrada[j])
			matriz.append(aux)
			aux = []
		for i in range(32):
			print(str(matriz[i]).replace(', ', ''))
		print(res, '=>',int(num[0] - 1))
		return int(num[0] - 1)

The Tkinter module (“Tk interface”) is the standard Python interface to the Tk GUI toolkit. Both Tk and Tkinter are available on most Unix platforms, as well as on Windows systems. (Tk itself is not part of Python; it is maintained at ActiveState.)

In [ ]:
from tkinter import *
import tkinter.messagebox as msg
import numpy as np

class Ventana(Frame):

	def __init__(self, master = None):
		super().__init__(root)
		self.master = master
		self.red = Red(946, 1000)
		self.inicializar()
		self.coordenadas = []

	def inicializar(self):
		self.master.title("Identificador de digitos")
		self.master.resizable(0,0)
		self.grid(row=0,column=0)
		self.matriz()
		
		btnEntrenar = Button(self, text="Entrenar Red", height=3, bg='yellow', fg='blue', highlightbackground='#3E4149', command=self.entrenar)
		btnEntrenar.grid(columnspan = 11, sticky = W + E + N + S,row = 32, column = 0)
		
		btnReiniciar = Button(self, text="Reiniciar Matriz", height=3, bg='red', highlightbackground='#3E4149', command=self.reiniciar)
		btnReiniciar.grid(columnspan = 10, sticky = W + E + N + S,row = 32, column = 11)
		
		btnPredecir = Button(self, text="Predecir Numero", height=3, bg='blue', fg='yellow', highlightbackground='#3E4149', command=self.predecir)
		btnPredecir.grid(columnspan = 11, sticky = W + E + N + S,row = 32, column = 21)

	def  entrenar(self):
		self.red.entrenar()

	def predecir(self):
		matriz = self.generarMatriz(32, self.coordenadas)
		numero = np.ravel(np.matrix(matriz))
		prediccion = self.red.predecir(numero)
		msg.showinfo("Resultado", "El Numero que usted dibujo es: " + str(prediccion))
		#for i in range(len(matriz)):
		#	print(matriz[i])

	def matriz(self):
		self.btn = [[0 for x in range(32)] for x in range(32)] 
		for x in range(32):
			for y in range(32):
				self.btn[x][y] = Button(self, bg='black', command=lambda x1=x, y1=y: self.seleccionar(x1,y1))
				self.btn[x][y].grid(column = x, row = y)

	def generarMatriz(self, n, coordenadas):
		matriz = []
		for i in range(n):
			matriz.append([0 for j in range(n)])

		for i in range(len(coordenadas)):
			x, y = coordenadas[i]
			matriz[y][x] = 1
		return matriz

	def seleccionar(self, x, y):
		self.btn[x][y].config(bg = "white")
		self.coordenadas.append((x, y))
		
	def reiniciar(self):
		self.matriz()
		self.coordenadas = []

if __name__ == '__main__':
	root = Tk()
	ventana = Ventana(root)
	root.mainloop()

/home/carlos/.conda/envs/cardosoSimulacion/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


[00000000111111111111111100000000]
[00000001111111111111111110000000]
[00000001111111111111111110000000]
[00000001111111111111111110000000]
[00000001111111111111110000000000]
[00000001111111111100000000000000]
[00000011111111000000000000000000]
[00000011111100000000000000000000]
[00000011111000000000000000000000]
[00000011111000000000000000000000]
[00000111111100000000000000000000]
[00000111111111000000000000000000]
[00000111111111110000000000000000]
[00000111111111111000000000000000]
[00000111111111111100000000000000]
[00000011111111111100000000000000]
[00000001100111111110000000000000]
[00000000000000111110000000000000]
[00000000000000111110000000000000]
[00000000000000111111000000000000]
[00000000000000011111000000000000]
[00000000000000001111100000000000]
[00000000000000001111100000000000]
[00000000000000001111100000000000]
[00000000000000001111100000000000]
[00000000000000001111100000000000]
[00000000110000111111000000000000]
[00000001111111111111000000000000]
[0000000111111111111

### Video

* Explanatory video of the coding source of the training of the network <br /><br /> [![Video of network training](https://img.youtube.com/vi/ybgVXcwiSCM/0.jpg)](https://youtu.be/ybgVXcwiSCM)<br />